In [1]:
# List of library and resources needed
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

#nltk.download('stopwords')
#nltk.download('punkt')

In [14]:
stop_words = set(stopwords.words('english'))

# Function to clean sentences using regex
def clean_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove non-alphanumeric characters using regex
    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [15]:
# Read the CSV file containing cleaned sentences and labels
data = pd.read_csv('combined_data_with_labels.csv')

# Extract sentences from the loaded data
combined_sentences = data['sentence']

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on the sentences and transform them into TF-IDF vectors
tfidf_vectors = tfidf_vectorizer.fit_transform(combined_sentences)

# Convert TF-IDF vectors to a dense representation (array)
tfidf_vectors_array = tfidf_vectors.toarray()

# Print the shape of the TF-IDF vectors array
print("Shape of TF-IDF Vectors Array:", tfidf_vectors_array.shape)

# Print the TF-IDF vectors array to verify
print("TF-IDF Vectors Array:", tfidf_vectors_array)

Shape of TF-IDF Vectors Array: (610, 1704)
TF-IDF Vectors Array: [[0.         0.         0.35491017 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [16]:
# Load the combined cleaned data with labels
combined_data = pd.read_csv('combined_data_with_labels.csv')

# Extract sentences and labels from the loaded data
sentences = combined_data['sentence']
labels = combined_data['label']

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on the sentences and transform them into TF-IDF vectors
tfidf_vectors = tfidf_vectorizer.fit_transform(sentences)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors, labels, test_size=0.3, random_state=94)

# Initialize the logistic regression model
logistic_regression_model = LogisticRegression()

# Train the model on the training data
logistic_regression_model.fit(X_train, y_train)

# Predict labels for the test data
y_pred = logistic_regression_model.predict(X_test)

# Calculate accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9398907103825137


In [17]:
# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       1.00      0.84      0.91        25
           1       0.98      0.85      0.91        47
           2       0.92      1.00      0.96       111

    accuracy                           0.94       183
   macro avg       0.96      0.90      0.93       183
weighted avg       0.94      0.94      0.94       183



In [18]:
# Save the trained model to a file
joblib.dump(logistic_regression_model, 'TF_IDF_gt_dt_et_regression_model.pkl')

['TF_IDF_gt_dt_et_regression_model.pkl']

In [19]:
# Load the saved model
loaded_model = joblib.load('TF_IDF_gt_dt_et_regression_model.pkl')

# Define a function to preprocess a single sentence
def preprocess_sentence(sentence):
    # Clean the sentence
    cleaned_sentence = clean_sentence(sentence)
    # Transform the cleaned sentence into TF-IDF vector
    tfidf_vector = tfidf_vectorizer.transform([cleaned_sentence])
    # Convert TF-IDF vector to a dense representation (array)
    tfidf_vector_array = tfidf_vector.toarray()
    return tfidf_vector_array

# Define a function to predict labels for a single sentence
def predict_label(sentence):
    # Preprocess the sentence
    preprocessed_sentence = preprocess_sentence(sentence)
    # Predict label using the loaded model
    prediction = loaded_model.predict(preprocessed_sentence)
    return prediction[0]

# Get user input for the sentence to be tested
user_sentence = input("Enter a sentence to be tested: ")

# Predict the label for the user input sentence
predicted_label = predict_label(user_sentence)

# Map predicted label to corresponding category
category = ""
if predicted_label == 1:
    category = "Displacement"
elif predicted_label == 0:
    category = "Energy"
else:
    category = "Not displacement or energy"

print("Predicted Category:", category)


Enter a sentence to be tested: the man is full of kinetic
Predicted Category: Not displacement or energy
